# Тестовое задание VK. Когорты и конверсия

## Необходимые библиотеки

In [4]:
import pandas as pd


import numpy as np
from datetime import datetime, timedelta
import random

## Загрузка данных

In [469]:
# Параметры данных
num_users = 100
num_products = 9  # 9 продуктов + null = 10 уникальных значений
events = ['launch', 'register', 'pageVisit', 'download', 'buy', 'update']
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 6, 1)

# Разделяем пользователей на две группы
user_ids = list(range(1, num_users + 1))
random.shuffle(user_ids)
update_users = set(user_ids[:num_users//2])  # Ровно половина пользователей

# Генерация данных
data = []
products = [f'product_{i}' for i in range(1, num_products + 1)]  # без None

for user_id in range(1, num_users + 1):
    # Определяем, будет ли у пользователя событие update
    user_has_update = user_id in update_users
    
    # Определяем общее количество событий для пользователя (от 1 до 1000)
    total_events = random.randint(1, 1000)
    
    # Генерация первого launch
    base_time = start_date + timedelta(days=random.randint(0, (end_date - start_date).days - 30))
    first_launch_time = base_time + timedelta(minutes=random.randint(0, 1439))
    data.append([user_id, 'launch', first_launch_time, None])
    events_generated = 1
    
    # Если у пользователя только 1 событие - завершаем
    if events_generated >= total_events:
        continue
    
    # Добавляем register через 1-60 секунд после первого launch
    register_time = first_launch_time + timedelta(seconds=random.randint(1, 60))
    data.append([user_id, 'register', register_time, None])
    events_generated += 1
    
    # Если у пользователя только 2 события - завершаем
    if events_generated >= total_events:
        continue
    
    # Дополнительные launch (от 0 до 20% от общего числа событий)
    max_additional_launches = min(int(total_events * 0.2), 20)
    for _ in range(random.randint(0, max_additional_launches)):
        additional_launch_time = register_time + timedelta(hours=random.randint(1, 24))
        data.append([user_id, 'launch', additional_launch_time, None])
        events_generated += 1
        if events_generated >= total_events:
            break
    
    if events_generated >= total_events:
        continue
    
    # Выбор продуктов для пользователя (1-5 продуктов)
    user_products = random.sample(products, random.randint(1, min(5, total_events - events_generated)))
    
    # Распределяем оставшиеся события между продуктами
    remaining_events = total_events - events_generated
    events_per_product = [1] * len(user_products)  # Каждому продукту минимум 1 pageVisit
    remaining_events -= len(user_products)
    
    # Распределяем оставшиеся события
    for i in range(remaining_events):
        events_per_product[random.randint(0, len(user_products) - 1)] += 1
    
    for product, product_events in zip(user_products, events_per_product):
        # Первый pageVisit для продукта
        page_visit_time = register_time + timedelta(days=random.randint(1, 20))
        if page_visit_time > end_date:
            continue
            
        # Добавляем launch за 1 минуту до pageVisit
        launch_time = page_visit_time - timedelta(seconds=random.randint(1, 60))
        data.append([user_id, 'launch', launch_time, None])
        data.append([user_id, 'pageVisit', page_visit_time, product])
        events_generated += 2
        
        # Оставшиеся события для этого продукта
        for _ in range(product_events - 1):
            action_time = page_visit_time + timedelta(seconds=random.randint(1, 60))
            
            # Для пользователей без update выбираем только между download и buy
            if user_has_update:
                action = random.choice(['download', 'buy', 'update'])
            else:
                action = random.choice(['download', 'buy'])
            
            data.append([user_id, action, action_time, product])
            events_generated += 1
            
            if events_generated >= total_events:
                break
        if events_generated >= total_events:
            break

# Создаем DataFrame
df = pd.DataFrame(data, columns=['userId', 'eventName', 'time', 'product'])

# Сортируем по user_id и времени
df = df.sort_values(['userId', 'time']).reset_index(drop=True)

# Проверка требований
print(f"Уникальных userId: {df['userId'].nunique()}")
print(f"Уникальных eventName: {df['eventName'].nunique()}")
print(f"Минимальное время: {df['time'].min()}")
print(f"Максимальное время: {df['time'].max()}")
print(f"Уникальных product: {df['product'].nunique()} (включая None)")

# Проверка распределения пользователей с update
users_with_update = df[df['eventName'] == 'update']['userId'].nunique()
print(f"\nПользователей с событиями update: {users_with_update} (должно быть {num_users//2})")

# Проверка распределения количества событий по пользователям
user_event_counts = df.groupby('userId').size()
print("\nРаспределение количества событий по пользователям:")
print(f"Минимальное количество событий: {user_event_counts.min()}")
print(f"Максимальное количество событий: {user_event_counts.max()}")
print(f"Среднее количество событий: {user_event_counts.mean():.1f}")

# Проверка всех условий
def check_all_conditions(df):
    errors = {
        'register_after_first_launch': 0,
        'register_time_diff': 0,
        'launch_before_page_visit': 0,
        'page_visit_time_diff': 0,
        'page_visit_before_actions': 0,
        'actions_time_diff': 0,
        'update_distribution': 0
    }
    
    # Проверка распределения update
    users_with_update = df[df['eventName'] == 'update']['userId'].nunique()
    if users_with_update != num_users // 2:
        errors['update_distribution'] = 1
    
    for user_id in df['userId'].unique():
        user_df = df[df['userId'] == user_id].sort_values('time')
        
        # Проверка register после первого launch
        launches = user_df[user_df['eventName'] == 'launch']
        registers = user_df[user_df['eventName'] == 'register']
        
        if len(registers) > 1:
            errors['register_after_first_launch'] += 1
        elif len(registers) == 1:
            first_launch = launches.iloc[0]['time']
            register = registers.iloc[0]['time']
            if (register - first_launch).total_seconds() > 60:
                errors['register_time_diff'] += 1
        
        # Проверка launch перед pageVisit
        page_visits = user_df[user_df['eventName'] == 'pageVisit']
        for _, pv_row in page_visits.iterrows():
            prev_events = user_df[user_df['time'] < pv_row['time']]
            if prev_events.empty or prev_events.iloc[-1]['eventName'] != 'launch':
                errors['launch_before_page_visit'] += 1
            elif (pv_row['time'] - prev_events.iloc[-1]['time']).total_seconds() > 60:
                errors['page_visit_time_diff'] += 1
            
            # Проверка действий после pageVisit
            next_events = user_df[(user_df['time'] > pv_row['time']) & 
                                (user_df['product'] == pv_row['product']) &
                                (user_df['eventName'].isin(['download', 'buy', 'update']))]
            
            for _, action_row in next_events.iterrows():
                if (action_row['time'] - pv_row['time']).total_seconds() > 60:
                    errors['actions_time_diff'] += 1
    
    return errors

errors = check_all_conditions(df)
print("\nПроверка условий:")
print(f"- Ровно у половины пользователей есть update: {'OK' if errors['update_distribution'] == 0 else 'ERROR'}")
print(f"- Не более одного register на пользователя: {errors['register_after_first_launch']} ошибок")
print(f"- Register через ≤1 минуты после первого launch: {errors['register_time_diff']} ошибок")
print(f"- Перед каждым pageVisit есть launch: {errors['launch_before_page_visit']} ошибок")
print(f"- Launch перед pageVisit с разницей ≤1 минуты: {errors['page_visit_time_diff']} ошибок")
print(f"- PageVisit перед download/buy/update: 0 ошибок (гарантировано структурой)")
print(f"- Download/buy/update через ≤1 минуты после pageVisit: {errors['actions_time_diff']} ошибок")

# Сохранение в CSV
df.to_csv('user_events_dataset.csv', index=False)
print("\nДанные сохранены в user_events_dataset.csv")

Уникальных userId: 100
Уникальных eventName: 6
Минимальное время: 2024-02-01 01:12:00
Максимальное время: 2024-05-21 05:09:46
Уникальных product: 9 (включая None)

Пользователей с событиями update: 48 (должно быть 50)

Распределение количества событий по пользователям:
Минимальное количество событий: 4
Максимальное количество событий: 997
Среднее количество событий: 514.5

Проверка условий:
- Ровно у половины пользователей есть update: ERROR
- Не более одного register на пользователя: 0 ошибок
- Register через ≤1 минуты после первого launch: 0 ошибок
- Перед каждым pageVisit есть launch: 0 ошибок
- Launch перед pageVisit с разницей ≤1 минуты: 0 ошибок
- PageVisit перед download/buy/update: 0 ошибок (гарантировано структурой)
- Download/buy/update через ≤1 минуты после pageVisit: 0 ошибок

Данные сохранены в user_events_dataset.csv


## Расчет когорт (Python)

In [17]:
df = pd.read_csv("user_events_dataset.csv")
df.head()

,userId,eventName,time,product
0,1,launch,2024-02-02 06:04:00,NaN
1,1,register,2024-02-02 06:04:44,NaN
2,1,launch,2024-02-06 06:04:30,NaN
3,1,pageVisit,2024-02-06 06:04:44,product_4
4,1,download,2024-02-06 06:04:45,product_4


In [18]:
import pandas as pd

df['time'] = pd.to_datetime(df['time'])

# Расчет первой даты запуска приложения first_launch_time
users_first_launches = df[df['eventName'] == 'launch'].groupby('userId', as_index=False)['time'].min().rename({"time": "first_launch_time"}, axis=1)

# Берем пользователей, у которых первый запуск приложения был после 2024-03-01
users_first_launches = users_first_launches[users_first_launches['first_launch_time'] >= '2024-03-01'].copy()

# Определение недели по дате первого запуска приложения week (чтобы отличать недельные когорты разных лет, учту год в определении когорты)
users_first_launches.loc[:, "week"] = users_first_launches['first_launch_time'].apply(lambda x: x.strftime("%Y-%W"))

# Статистики по когортам (week), где users - общее количество пользователей в когорте
cohorts = users_first_launches.groupby('week', as_index=False)['userId'].nunique().rename({"userId": "users"}, axis=1)

# Соединение users_first_launches и исходного датасета df, чтобы получить действия всех пользователей, первая дата захода в приложение которых больше 2024-03-01,
# с номером когорты и датой первого запуска приложения
df = df.merge(users_first_launches, on='userId', how='inner')

# Расчет количества пользователей из когорты, совершивших хотя бы одно обновление приложения
# в течение 14 дней после первого запуска
stats = df[(df['eventName'] == "update") &\
        (df['time'] > df['first_launch_time']) &\
        (df['time'] <= df['first_launch_time'] + pd.to_timedelta('14d'))]\
        .groupby("week", as_index=False).userId.nunique()\
        .rename({"userId": "updaters"}, axis=1)

# Расчет доли пользователей из когорты
result = stats.merge(cohorts)
result.loc[:, "CR"] = result.loc[:, "updaters"]/result.loc[:, "users"]
result.drop("updaters", axis=1, inplace=True)

print(result)

      week  users        CR
0  2024-09      2  0.500000
1  2024-10      6  0.166667
2  2024-11      6  0.333333
3  2024-12      9  0.555556
4  2024-13      5  0.400000
5  2024-14      5  0.400000
6  2024-15      4  0.500000
7  2024-16      4  0.500000
8  2024-17      8  0.500000
9  2024-18      6  0.500000


## Расчет когорт (SQL)

In [19]:
import sqlite3

In [20]:
conn = sqlite3.connect("test_VK_2.db")
cursor = conn.cursor()

In [492]:
df = pd.read_csv("user_events_dataset.csv")
df.head()

,userId,eventName,time,product
0,1,launch,2024-02-02 06:04:00,NaN
1,1,register,2024-02-02 06:04:44,NaN
2,1,launch,2024-02-06 06:04:30,NaN
3,1,pageVisit,2024-02-06 06:04:44,product_4
4,1,download,2024-02-06 06:04:45,product_4


In [15]:
df['time'] = pd.to_datetime(df['time'])

In [494]:
df.head()

,userId,eventName,time,product
0,1,launch,2024-02-02 06:04:00,NaN
1,1,register,2024-02-02 06:04:44,NaN
2,1,launch,2024-02-06 06:04:30,NaN
3,1,pageVisit,2024-02-06 06:04:44,product_4
4,1,download,2024-02-06 06:04:45,product_4


In [495]:
df.to_sql("table", conn, index=False)

51454

In [23]:
text = """
    /* Используется диалект SQLite */
    /* Таблица с расчетом когорты для каждого пользователя и кол-ву пользователей в когорте*/
    WITH first_lt as (
        SELECT userId,
            MIN(time) as first_launch_time,
            STRFTIME('%Y-%W', MIN(time)) AS week -- Необходимо для того, чтобы недельные когорты разных лет отличались
        FROM "table"
        WHERE eventName == 'launch'
        GROUP BY userId
        HAVING MIN(time) >= '2024-03-01'
    ),
    cohorts AS (
        SELECT userId,
            first_launch_time,
            week,
            COUNT(userID) OVER (PARTITION BY week) AS users
        FROM first_lt
    )
    SELECT cohorts.week as week, cohorts.users as users,
        CAST(COUNT(DISTINCT "table".userId) AS REAL) / cohorts.users as 'CR'
    FROM "table" INNER JOIN cohorts ON "table".userId = cohorts.userID
    WHERE eventName == 'update' AND time > first_launch_time AND time <= DATETIME(first_launch_time, '+14 days')
    GROUP BY cohorts.week;
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res.columns = ["week", "users", "CR"]
res

,week,users,CR
0,2024-09,2,0.500000
1,2024-10,6,0.166667
2,2024-11,6,0.333333
3,2024-12,9,0.555556
4,2024-13,5,0.400000
5,2024-14,5,0.400000
6,2024-15,4,0.500000
7,2024-16,4,0.500000
8,2024-17,8,0.500000
9,2024-18,6,0.500000


In [24]:
res == result

,week,users,CR
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True
7,True,True,True
8,True,True,True
9,True,True,True


In [482]:
conn = sqlite3.connect("test_VK_2.db")
cursor = conn.cursor()

cursor.execute("CREATE TABLE result (week TEXT PRIMARY KEY, users INT, CR REAL)")

In [483]:
conn.commit()

In [484]:
conn = sqlite3.connect("test_VK_2.db")
cursor = conn.cursor()

cursor.execute(f"INSERT INTO result (week, users, CR) {text}")
conn.commit()
conn.close()


In [485]:
conn = sqlite3.connect("test_VK_2.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM result")
users = cursor.fetchall()

# Выводим результаты
for user in users:
  print(user)

('09th week of 2024', 2, 0.5)
('10th week of 2024', 6, 0.16666666666666666)
('11th week of 2024', 6, 0.3333333333333333)
('12th week of 2024', 9, 0.5555555555555556)
('13th week of 2024', 5, 0.4)
('14th week of 2024', 5, 0.4)
('15th week of 2024', 4, 0.5)
('16th week of 2024', 4, 0.5)
('17th week of 2024', 8, 0.5)
('18th week of 2024', 6, 0.5)


In [486]:

cursor.execute("PRAGMA table_info('result') ")
users = cursor.fetchall()

# Выводим результаты
for user in users:
  print(user)

(0, 'week', 'TEXT', 0, None, 1)
(1, 'users', 'INT', 0, None, 0)
(2, 'CR', 'REAL', 0, None, 0)


In [527]:
a = pd.DataFrame({"id": [1, 2, 3], "chief_id":[2, 3, None], "name": ['a', 'b', 'c'], 'salary': [1000, 2000, 3000]})

In [528]:
a.merge(a, left_on='id', right_on='chief_id', how='right')

,id_x,chief_id_x,name_x,salary_x,id_y,chief_id_y,name_y,salary_y
0,2.0,3.0,b,2000.0,1,2.0,a,1000
1,3.0,NaN,c,3000.0,2,3.0,b,2000
2,NaN,NaN,NaN,NaN,3,NaN,c,3000


In [531]:
a.to_sql("employees_1", conn, index=False)

3

In [532]:
text = """
    select a.* from employees_1 a left join employees_1 b on b.id = a.chief_id AND a.salary < b.salary
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0,1,2,3
0,1,2.0,a,1000
1,2,3.0,b,2000
2,3,NaN,c,3000


In [534]:
text = """
    select a.* from employees_1 a join employees_1 b on b.id = a.chief_id AND a.salary > b.salary
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

""


In [536]:
a = pd.DataFrame({"value": [1, 2, 4, 4, 5, None]})
b = pd.DataFrame({"value": [1, 3, 4, 4, 5, 5, None]})

In [538]:
a.to_sql("tabA", conn, index=False)

6

In [540]:
b.to_sql("tabbB", conn, index=False)

7

In [548]:
text = """
    select * from tabA inner join tabbB on tabA.value = tabbB.value
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0,1
0,1.0,1.0
1,4.0,4.0
2,4.0,4.0
3,4.0,4.0
4,4.0,4.0
5,5.0,5.0
6,5.0,5.0
